# Árboles de Decisión

Tecnológico de Costa Rica

Inteligencia Artificial

Proyecto Corto 3-4 

Estudiantes:

    Kahho Chen Li
    Erick Hernández López
    Siul Mongalo Monge
    
    

## ¿Qué son?
Los árboles de decisión son algoritmos de aprendizaje supervisado usados para realizar clasificaciones, asignándolos a aprendizajes basados en la información que poseen con el uso de procesos matemáticos que determinan la ganancia, es decir, un árbol de decisión es una forma gráfica y analítica de representar todos los eventos que pueden surgir a partir de una decisión asumida en cierto momento.

El principal objetivo de estos árboles está en encontrar esas características descriptivas que llevan la mayor "información" con respecto a la característica objetivo, con el fin de dividir el conjunto de datos a lo largo de los valores de estas características y así los valores de las características objetivo sean los más relacionables para los subconjuntos de datos generados.

## Explicación del algoritmo

Los algoritmos de los árboles de decisiones se basan en diferentes secciones:

1. **Lectura de Archivo .CSV**

Esta lectura consiste en cargar las filas del conjunto de datos en memoria, asignándolas a una estructura de datos, en este caso se usa listas, para poder mandar los datos limpios a las otras funciones del proceso. 

2. **Crear árbol de decisón**

Esta sección consiste en armar el árbol de decisión en base a los ejemplos que trae el conjunto de datos, para ésto se necesita que la función reciba los datos (lista de listas), los atributos, y el atributo (target) que vamos a querer predecir. 

Esta función toma diferentes caminos dependiendo de la situación actual de los valores:

° Si los datos están vacíos retorna el default.

° Si todos los ejemplos tienen las misma clasificación retorna esa clasificación.

° Sino, elige el siguiente mejor atributo, verifica si hay nodos faltantes, crea un nuevo nodo para cada posible valor del mejor atributo por medio de un ciclo de los valores actuales para al final asignarle el nodo al árbol original.

Dentro de esta función principal, se llaman a otras funciones que completan los procesos de ésta. Entre las funciones que se conectan están:

* *Frecuencia* => Es la función que retorna el diccionario con la frecuencia de los datos.
* *Valor Mayoría* => Es la función que busca los atributos más comunes.
* *Ganancia de Información* => Es la función que calcula la frecuecia y la entropía de los atributos con el propósito de determinar si hay que dividir con dicho atributo.
* *Obtener Valores* => Es la función que retorna una lista con todos los posibles valores de un atributo.
* *Obtener Filas Válidas* => Es la función que retorna las filas que a ese nivel del árbol siguen siendo válidas.



3. **Predecir en el árbol**

Esta sección crea una lista de llaves y crea un sub-árbol en el cual se verifican los valores en cada rama del árbol en general para determinar una predicción.

### Conjunto de Datos a usar en el árbol

In [20]:
import pandas as pd
dataframe = pd.read_csv("DataTree.csv")
dataframe

,VisitaFamiliar,Clima,Deporte,Futbol,Dinero,Dormir
0,Si,Soleado,No,No,Si,Si
1,No,Soleado,Si,Si,No,No
2,No,Ventoso,Si,Si,No,No
3,Si,Lluvioso,No,No,Si,Si
4,No,Soleado,Si,MM,Si,No
5,No,Lluvioso,Si,No,No,Si
6,No,Ventoso,No,No,Si,No
7,No,Ventoso,Si,MM,No,Si
8,Si,Soleado,Si,No,No,Si
9,No,Soleado,Si,No,Si,Si


### Algoritmo del árbol de decisión

In [26]:
# %load DecTreeFinal.py
import math
import random

'''Fnción que retorna el diccionario con la frecuencia de los datos'''
def frecuencia(datos, frecuencias, index):
    #recorre los datos para verificar que el indice de la fila esté en las frecuencias
    for fila in datos:
        if (fila[index] in frecuencias):
            frecuencias[fila[index]] += 1.0 #si está le suma uno
        else:
            frecuencias[fila[index]] = 1.0  #sino, le asigna uno
    return frecuencias

'''Funcion que busca los atributos mas comunes'''
def valor_mayoria(atributos, datos, target):
    valor_frecuencia = {}
    index = atributos.index(target) #Busca el target
    valor_frecuencia = frecuencia(datos, valor_frecuencia, index) #Calcula frecuencia de datos
    mayoria_A = True #el default de la variable es true
    maximo = 0.0 #asigna el mínimo a la variable
    mayor = ""
    #hace un ciclo que recorre las llaves de los valores de frecuencia 
    for llave in valor_frecuencia.keys():
        if valor_frecuencia[llave] > maximo:
            maximo = valor_frecuencia[llave]
            mayor = llave
            mayoria_A = True
        elif valor_frecuencia[llave] == maximo:
            mayoria_A = False

    #Si no existe mayoria se retorna un random
    if not mayoria_A:
        valores = list(valor_frecuencia.keys())
        rand = random.choice(valores)
        return rand

    return mayor

'''Funcion que calcula la entropia de los datos
   Retorna el valor de la entropia'''
def calcular_entropia(atributos, datos, Attr):
    entropia = 0.0 #le asigna el valor mínimo a la entropía
    valor_frecuencia = {}
    indice = 0 #selecciona index inicial
    
    for valor in atributos:
        if (Attr == valor):
            break
        indice += 1

    #calcula la frecuencia de los datos con la función frecuencia
    valor_frecuencia = frecuencia(datos, valor_frecuencia, indice)

    # Calcula la entropia recorriendo los valores de las frecuencias
    for freq in valor_frecuencia.values():
        #la fórmula de la entropía se calcula con la sumatoria de la
        #la frecuencia entre el total de los datos por el log2 de lo mismo 
        entropia += (-freq/len(datos)) * math.log(freq/len(datos), 2)


    return entropia

''' Retorna la ganancia de informacion del atributo
    Se calcula esa ganancia para determinar si hay que dividir con este atributo'''
def ganancia_informacion(atributos, datos, attr, targetAttr):
    valor_frecuencia = {}
    subset_entropia = 0.0
    
    indice = atributos.index(attr) #index del atributo

    # Se calcula las veces que aparece cada valor del atributo al que se le está calculando ganancia 
    valor_frecuencia = frecuencia(datos, valor_frecuencia, indice)

    # Se calcula la entropia para cada uno de los subgrupos 
    for valor in valor_frecuencia.keys():
        valor_probabilidad = valor_frecuencia[valor] / sum(valor_frecuencia.values())
        dataSubset =  []

        for fila in datos:
            if fila[indice] == valor:
                dataSubset.append(fila)
        #calculamos la entropía llamando a la función
        valor_entropia = calcular_entropia(atributos, dataSubset, targetAttr)
        subset_entropia += valor_probabilidad * valor_entropia #Resto

    entropia_total = calcular_entropia(atributos, datos, targetAttr)
    ganancia = entropia_total - subset_entropia #hace la resta de las entropías para la ganancia

    return ganancia

'''Función que escoge el mejor atributo
    Retorna ese mejor atributo con su ganancia'''
def escoger_informacion(datos, atributos, target):
    mejor = atributos[0] #asigna el primer atributo a la variable mejor 
    maxima_ganancia = 0
    for attr in atributos:
        if attr != target: 
            #si son diferentes calcula una nueva ganancia
            nueva_ganancia = ganancia_informacion(atributos, datos, attr, target) 
            if nueva_ganancia > maxima_ganancia:
                maxima_ganancia = nueva_ganancia
                mejor = attr
    if(maxima_ganancia > 1): maxima_ganancia = 1.0
    return mejor, maxima_ganancia

'''Retorna una lista con todos los posibles valores de un atributo'''
def obtener_valores(datos, atributos, attr):
    indice = atributos.index(attr)
    valores = []
    #ciclo que recorre las filas de los datos para ver si el indice está o no
    for fila in datos:
        if fila[indice] not in valores: 
            valores.append(fila[indice])#lo agrego a los valores si no se encontró
    
    return valores

'''Retorna las filas que a este nivel del arbol siguen siendo validas'''
def obtener_filas_validas(datos, atributos, mejor, val):
    filas_validas = [[]]
    indice = atributos.index(mejor)
    for valor in datos:
        if (valor[indice] == val):
            nueva_entrada = []

            #agrega el valor si no es el mejor
            for i in range(0,len(valor)):
                if(i != indice):
                    nueva_entrada.append(valor[i])
            filas_validas.append(nueva_entrada)
    filas_validas.remove([])
    return filas_validas

def crear_arbol(datos, atributos, target):
    
    datos = datos[:] #Copia del parametro data, no una referencia
    valores = [] 
    
    #Una lista con todos los valores actuales del target
    for fila in datos:
        valores.append(fila[atributos.index(target)])
    #Retorna cual atributo tiene más presencia, ese atributo será el default
    default = valor_mayoria(atributos, datos, target) 
    
    # Si el dataset esta limpio retorna el default value. 
    # Se verifica si aun quedan atributos, se resta - 1 para no tomar en cuenta el atributo target

    #Si los datos están vacíos
    if not datos or (len(atributos) - 1) <= 0:
        return default
    # Si todos los ejemplos tienen la misma clasificación entonces retorne esa clasificación
    if valores.count(valores[0]) == len(valores):
        return valores[0]
    else:
        # Elija el siguiente mejor atributo
        mejor, maxima_ganancia = escoger_informacion(datos, atributos, target)

        # Creamos un nuevo nodo con el mejor atributo
        arbol = {mejor:{'GanInfo':maxima_ganancia, 'ValMay':default}}

        # Verificamos si hay nodos que falten de agregar
        valores_actuales = obtener_valores(datos, atributos, mejor)

        # Creamos un nuevo nodo por cada posible valor del mejor atributo
        for valor in valores_actuales:
            filas_validas = obtener_filas_validas(datos, atributos, mejor, valor) #Retorna todos los ejemplos para un valor del mejor atributo
            nuevo_atributo = atributos[:] #Hace una copia de los atributos para no modificar los originales
            nuevo_atributo.remove(mejor) #Quita el mejor atributo
            sub_arbol = crear_arbol(filas_validas, nuevo_atributo, target) #Crea el nuevo nodo
            # Asigna el nuevo nodo
            arbol[mejor][valor] = sub_arbol
        
    return arbol

def prediccion_aux_arbol(arbol, input):
    llave_atributo = list(arbol.keys())[0]
    sub_arbol = arbol[llave_atributo]
    
    valor_entrada = input.get(llave_atributo,None)
    
    if(valor_entrada not in sub_arbol):
        # Es un nuevo caso no visto del branch
        return sub_arbol['ValMay']
    rama = sub_arbol[valor_entrada]

    if(type(rama) is dict):
        return prediccion_aux_arbol(rama, input)
    else:
        return rama

def prediccion_arbol_decision(arbol, input):
    arbol_temporal = arbol.copy()
    return prediccion_aux_arbol(arbol_temporal, input)



def construir_arbol_decision(nombre_archivo, target):
    archivo = open(nombre_archivo)
    datos = [[]]

    for linea in archivo:
        linea = linea.strip("\r\n")
        datos.append(linea.split(','))
    datos.remove([])
    atributos = datos[0]
    datos.remove(atributos)
    #Recibe una lista de listas, los atributos, y el atributo que vamos a querer predecir  
    return crear_arbol(datos, atributos, target)

Dataset = 'DataTree.csv'
target = 'Deporte'
arbol = construir_arbol_decision(Dataset,target)

print("Versión del Árbol en llaves\n\n", arbol)

result = prediccion_arbol_decision(arbol, {'Dormir':'Si'})

print("Prediccion del resultado es: ",result)



Versión del Árbol en llaves

 {'Dinero': {'GanInfo': 0.3958156020033584, 'ValMay': 'Si', 'Si': {'VisitaFamiliar': {'GanInfo': 0.4199730940219749, 'ValMay': 'No', 'Si': 'No', 'No': {'Clima': {'GanInfo': 0.9182958340544896, 'ValMay': 'Si', 'Soleado': 'Si', 'Ventoso': 'No'}}}}, 'No': 'Si'}}
Prediccion del resultado es:  Si


In [27]:
import json

In [28]:
jTree = json.dumps(arbol, indent = 5)

In [29]:
print (jTree)

{
     "Dinero": {
          "GanInfo": 0.3958156020033584,
          "ValMay": "Si",
          "Si": {
               "VisitaFamiliar": {
                    "GanInfo": 0.4199730940219749,
                    "ValMay": "No",
                    "Si": "No",
                    "No": {
                         "Clima": {
                              "GanInfo": 0.9182958340544896,
                              "ValMay": "Si",
                              "Soleado": "Si",
                              "Ventoso": "No"
                         }
                    }
               }
          },
          "No": "Si"
     }
}


# Random Forest Algorithm

Random Forest es un algoritmo de clasificación supervisado. El algoritmo, como implica en su nombre, crea bosques (conjunto de árboles de decisión) y lo hace aleatoriamente, existe una relacion directa entre la cantidad de arboles y la precision del resultado que genera. La diferencia entre el algoritmo de un árbol de decisión y el de un Random Forest está en el proceso de la busqueda del nodo para hacer la separación, el algoritmo del Random Forest se separa aleatoriamente, mientras que en el de un árbol de decisión depende de ciertas reglas.

#### Algoritmo Pseudocódigo

1. Aleatoriamente selecciona **"K"** características de un conjunto de **m** características donde **k < m**.
2. Del conjunto de **K** características, se calcula el nodo **n** usando el mejor punto de división.
3. Se divide el nodo en **nodos hijos** usando *la mejor devisión*.
4. Repite los pasos 1 - 3 hasta una cantidad de nodos deseados.
5. Se construye el bosque repitiendo los pasos 1 - 4 **i** veces para crear **i** número de árboles.
- Para realizar una predicción se debe seguir los siguientes pasos:
1. Toma las características de prueba y usa las reglas establecidas para cada árbol de decisión para predecir y alamacenar el resultado objetivo.
2. Calcula los votos para cada objetivo predecido.
3. Considera los objetivos que tienen más votos como la **la predicción final**.



En la siguiente figura podemos ver un ejemplo de cómo seria al algoritmo de Random Forest. Tomado de la página web: https://medium.com/@williamkoehrsen/random-forest-simple-explanation-377895a60d2d

![title](EjemploRandomForest.png)

#### Ejemplos de usos prácticos

Este algoritmo puede ser usado en varias aplicaciones de la vida real, según el autor Saimadhu Polamuri en un articulo web (https://dataaspirant.com/2017/05/22/random-forest-algorithm-machine-learing/) Se puede ver en:

1) Bancario: Como en un banco se basa en la confianza del cliente, los datos de estos clientes son analizados profundamente para encontrar patrones de lealtad de los clientes. De esta manera pueden categorizar los clientes buenos y malos, estos son los que pueden tomar prestamo grandes cantidades de dinero y se pagan debidamente y los que no son rentable para el banco.

2) Medicina: Se usa generalmente para identificar la combinación correcta de los componentes para poder validar una medicina. Y a la misma vez, se puede utilizar para identificar enfermedades analizando los antecedentes del paciente.

3) Bolsa de valores: El algoritmo se utiliza para identificar el comportamiento de la bolsa de valores y las perdidas y ganancias de ciertas acciones.


#### Ventajas

- El algoritmo de Random Forest puede ser usada para problemas de clasificación y regresión.
- El clasificador de Random Forest puede manejar valores que no están presentes.
- Al tener muchos árboles, el clasificador no va a **overfit** el modelo.

#### Desventajas
- Aunque las aplicaciones del algoritmo es bastante sencillo y lo suficientemente rápido para ciertas situaciones, es un algoritmo que no es muy efectivo en tiempo real, ya que se requiere muchos arboles para poder realizar predicciones más precisas, esto implica más tiempo.
- Este algoritmo es una herramiente predictiva, es decir, no describe las relaciones que tiene las predicciones con los datos.
